In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter



In [2]:
# 讀取檔案
file_path = r"C:\Users\mindy\桌面\張茗溱.pdf"
# file_path = r"E:\腸易激.pdf"
loader = file_path.endswith(".pdf") and PyPDFLoader(file_path) or TextLoader(file_path)


In [3]:
# 選擇 splitter 並將文字切分成多個 chunk 
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0) 
texts = loader.load_and_split(splitter)


In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# 建立本地 db
# 建立本地 db
embeddings = HuggingFaceEmbeddings(model_name="shibing624/text2vec-base-chinese")
db = Chroma.from_documents(
    texts, 
    embeddings, 
    persist_directory="./chroma_db"
)
retriever = db.as_retriever(search_kwargs={"k": 2})

C:\Users\mindy\AppData\Local\Temp\ipykernel_6596\3861624617.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="shibing624/text2vec-base-chinese")
c:\Users\mindy\anaconda3\envs\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Setup the pipeline

from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="taide/TAIDE-LX-7B-Chat-4bit",
	filename="taide-7b-a.2-q4_k_m.gguf",
    n_ctx=4096,  # 增加上下文長度，預設通常是 512 或 2048
    max_tokens=3000
)

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from C:\Users\mindy\.cache\huggingface\hub\models--taide--TAIDE-LX-7B-Chat-4bit\snapshots\9063ed154144775841f3953b69b534c6e2d564d8\.\taide-7b-a.2-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.ro

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = """
根據以下檢索資料，請提供一個的回答，並補充必要的背景信息和實例。請確保回答全面且深入。
資料：
{context}
問題：
{question}
"""
prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)
question='張茗溱有哪些實務專案經驗'
retrieved_docs = retriever.get_relevant_documents(question)
context = "\n".join([doc.page_content[:1000] for doc in retrieved_docs])  

response = llm(f"{prompt.format(context=context, question=question)}", max_tokens=2048, temperature=0.8, stream=True)
for chunk in response:
    print(chunk["choices"][0]["text"], end="", flush=True)

C:\Users\mindy\AppData\Local\Temp\ipykernel_6596\3192943851.py:11: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(question)


In [2]:
!where python
!pip list

c:\Users\mindy\anaconda3\envs\rag\python.exe
C:\Users\mindy\anaconda3\python.exe
C:\Users\mindy\AppData\Local\Microsoft\WindowsApps\python.exe
Package                                  Version
---------------------------------------- ---------------
accelerate                               1.2.1
aiofiles                                 24.1.0
aiohappyeyeballs                         2.4.4
aiohttp                                  3.11.11
aiosignal                                1.3.2
annotated-types                          0.7.0
anyio                                    4.7.0
asgiref                                  3.8.1
asttokens                                3.0.0
async-timeout                            4.0.3
attrs                                    24.3.0
backoff                                  2.2.1
bcrypt                                   4.2.1
beautifulsoup4                           4.12.3
build                                    1.2.2.post1
cachetools                         

In [7]:
retrieved_docs 

[Document(metadata={'page': 0, 'source': 'C:\\Users\\mindy\\桌面\\張茗溱.pdf'}, page_content='張茗溱，畢業於國立東華大學資訊工程研究所，並持有國立嘉義大學應用數學\n系的學士學位。他擁有紮實的數學與資訊工程基礎，特別專注於機器學習與深\n度學習領域，並且在學術研究與實務專案中展現了優異的能力。 \n在碩士研究期間，張茗溱針對混淆資料的區分與再學習提出了一系列創新的方\n法，不僅提升了模型在 CIFAR-10 與 CIFAR-100 數據集上的準確率，還解決了\n資料集分布不均問題，為機器學習模型的設計與應用提供了新的思路。透過自\n定義的深度學習模型架構，他熟練掌握了 PyTorch 的使用，並展現了規劃實\n驗、分析結果以及有效解決問題的能力。此外，他對數據處理技術如 Pandas、\nNumPy，以及機器學習工具如 Scikit-learn、XGBoost、LightGBM 等也有深入了\n解。 \n張茗溱在學士期間即展現出卓越的創新能力，參與了多項具有實用性的研究專\n題。他的畢業專題研究「利用 AI 預測番茄成熟度」中，結合了 K-means 和 \nSVM 技術，成功解決了數據缺失處理的挑戰，並榮獲嘉義大學理工學院創意專\n題競賽的佳作。在學術之外，他亦積極投入實務專案，如與同學共同開發了一')]